In [16]:
import pandas as pd
import numpy as np

def cleanScenario(df):
    
    df_new = df[["ts", "icao24", "df", "tc", "callsign", "bds20_callsign", "lat_deg", "lon_deg", "altitude", "bds05_altitude", "groundspeed", "bds50_groundspeed", "wake_vortex"]]
    
    WAKE_VORTEX_TRANSLATION = {
        '<7000kg': 'Light (less than 7000 kg)',
        '<34,000kg': 'Medium 1 (between 7000 kg and 34000 kg)',
        '<136,000kg': 'Medium 2 (between 34000 kg to 136000 kg)',
        'Heavy': 'Heavy (larger than 136000 kg)',
        'High vortex': 'High vortex aircraft',
        'Rotorcraft': 'Rotorcraft',
        'Obstruction': 'Ground obstruction',
        'Surface emergency vehicle': 'Surface emergency vehicle',
    }
    
    df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)
    df_new = df_new.drop(columns=["wake_vortex"])

    # Cambiamos el formato de al columna timestamp
    df_new["ts"] = pd.to_datetime(df_new["ts"], unit='ms') # para que se pueda leer también con spark
    
    # Pasamos el typecode y downlink format a enteros
    df_new["tc"] = df_new["tc"].astype("Int64")
    df_new["df"] = df_new["df"].astype("Int64")
    
    # Combinamos las dos columnas de callsign en una sola
    df_new["Callsign"] = df_new["callsign"].fillna(df_new["bds20_callsign"])
    df_new = df_new.drop(columns=["callsign", "bds20_callsign"])
    
    # Combinamos las dos columnas de speed en una sola
    df_new["Speed"] = df_new["groundspeed"].fillna(df_new["bds50_groundspeed"])
    df_new = df_new.drop(columns=["groundspeed", "bds50_groundspeed"])
    
    # Combinamos las dos columnas de altitude en una sola
    df_new["Altitude (ft)"] = df_new["altitude"].fillna(df_new["bds05_altitude"])
    df_new = df_new.drop(columns=["altitude", "bds05_altitude"])
    
    df_new = df_new.rename(columns={"ts": "Timestamp (date)", "icao24": "ICAO", "df": "Downlink Format", "tc": "Typecode",
                                     "lat_deg": "lat", "lon_deg": "lon"})

    # Flight Status basado en Downlink Format y Typecode
    conditions_fs = [
        (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(5, 9))), # TC 5-8 (Tierra)
        (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(9, 23))), # TC 9-22 (Airborne) 
    ]
    
    results_fs = [
        'on-ground',
        'airborne',
    ]
    
    df_new["Flight status"] = np.select(conditions_fs, results_fs, default='NaN') 

    return df_new

In [18]:
import pandas as pd
import os
from pathlib import Path

# Configurar para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ruta sde entrada y salida
ruta_escenarios_raw = Path("../data/final_scenarios/raw")
ruta_escenarios_clean = Path("../data/final_scenarios/clean")
ruta_escenarios_clean.mkdir(parents=True, exist_ok=True)

# Iterar sobre todos los archivos .parquet en la carpeta raw
for parquet_file in ruta_escenarios_raw.glob("*.parquet"):
    print(f"Procesando {parquet_file.name}")
    df = pd.read_parquet(parquet_file)
    df_clean = cleanScenario(df)
    output_path = ruta_escenarios_clean / parquet_file.name
    df_clean.to_parquet(output_path, index=False)
    print(f"Guardado en {output_path}")


Procesando scenario_064.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)
/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)


Guardado en ../data/final_scenarios/clean/scenario_064.parquet
Procesando scenario_074.parquet
Guardado en ../data/final_scenarios/clean/scenario_074.parquet
Procesando scenario_006.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)


Guardado en ../data/final_scenarios/clean/scenario_006.parquet
Procesando scenario_016.parquet


...

Guardado en ../data/final_scenarios/clean/scenario_061.parquet
Procesando scenario_119.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)


Guardado en ../data/final_scenarios/clean/scenario_119.parquet
Procesando scenario_013.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)
/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_11419/2679907221.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)


Guardado en ../data/final_scenarios/clean/scenario_013.parquet
Procesando scenario_003.parquet
Guardado en ../data/final_scenarios/clean/scenario_003.parquet
